# 3. Feature Detection and Description



In [1]:
import numpy as np 
import cv2 as cv 


## 3.1 Harris Corner Detection

**Goal**
- Harris Corner Detection algorithm
- cv2.cornerHarris(), cv2.cornerSubPix()

### Theory

Corners are regions in the image with large variation in intensity in all the directions.
One early attempt to find these corners was done by Chris Harris & Mike Stephens in their paper A Combined Corner and Edge Detector in 1988.

The algorithm procedure is as follows:
- 1. Determine which windows produce very large variations in intensity when moved in both X and Y directions.
- 2. With each such window found, a score R is computed.
- 3. After applying a threshold to this score, important corners are selected & marked.

Find the difference in intensity for a displacement of $(u,v)$ in all directions.

$$
E(u,v) = \sum_{x,y} w(x,y)[I(x+u,y+v) - I(x,y)]^2
$$

where $I(x,y)$ is the intensity of the image at pixel $(x,y)$ and $w(x,y)$ is a window function. The window is either a rectangular window or a Gaussian window which gives weights to pixels underneath.

We have to maximize this function $E(u,v)$ for corner detection. That means, we have to maximize the second term. Applying Taylor Expansion to the second term, we get the score function.

The final equation after derivation is:
$$
E(u,v) \approx \begin{bmatrix} u & v \end{bmatrix} M \begin{bmatrix} u \\ v \end{bmatrix}
$$

where $M$ is a matrix of derivatives called **structure tensor** given by:

$$
M = \sum_{x,y} w(x,y) \begin{bmatrix} I_x I_x & I_x I_y \\ I_x I_y & I_y I_y \end{bmatrix}
$$

where $I_x$ and $I_y$ are image derivatives in $x$ and $y$ directions respectively.

Then, the score function is:

$$
R = det(M) - k(trace(M))^2
$$

where $det(M) = \lambda_1 \lambda_2$ and $trace(M) = \lambda_1 + \lambda_2$. $\lambda_1$ and $\lambda_2$ are the eigenvalues of $M$.

So, the values of $\lambda_1$ and $\lambda_2$ decide whether a region is corner, edge or flat.
- when $|R|$ is small, which happens when $\lambda_1$ and $\lambda_2$ are small, the region is flat.
- when $R<0$, which happens when $\lambda_1 >> \lambda_2$ or vice versa, the region is edge.
- when $R$ is large, which happens when $\lambda_1$ and $\lambda_2$ are large and $\lambda_1 \sim \lambda_2$, the region is a corner.

The result of Harris Corner Detection is a grayscale image with these scores. Thresholding for a suitable value gives the corner points.



### OpenCV Implementation

OpenCV has the function `cv2.cornerHarris()` for this purpose. Its arguments are:
- img - Input image, it should be grayscale and float32 type.
- blockSize - It is the size of neighbourhood considered for corner detection
- ksize - Aperture parameter of Sobel derivative used.
- k - Harris detector free parameter in the equation.


In [2]:
filename = 'messi5.jpg'
img = cv.imread(filename)
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

img_gray = np.float32(img_gray)
dst = cv.cornerHarris(img_gray, 2, 3, 0.04)

# result is dilated for marking the corners, not important
dst = cv.dilate(dst, None)

# threshold for an optimal value, it may vary depending on the image
img[dst > 0.01 * dst.max()] = [0, 0, 255]

cv.imshow('dst', img)
if cv.waitKey(0) & 0xff == 27:
    cv.destroyAllWindows()


Sometimes you may want to find the corners with maximum accuracy. OpenCV comes with a function `cv2.cornerSubPix()` which further refines the corners detected with sub-pixel accuracy. Below is an example. As usual, we need to find the harris corners first. Then we pass the centroids of these corners (There may be a bunch of pixels at a corner, we take their centroid) to refine them. Harris corners are marked in red pixels and refined corners are marked in green pixels. For this function, we have to define the criteria when to stop the iteration. We stop it after a specified number of iteration or a certain accuracy is achieved, whichever occurs first. We also need to define the size of neighbourhood it would search for corners.



In [3]:
filename = 'messi5.jpg'
img = cv.imread(filename)
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

img_gray = np.float32(img_gray)
dst = cv.cornerHarris(img_gray, 2, 3, 0.04)

# result is dilated for marking the corners, not important
dst = cv.dilate(dst, None)
ret, dst = cv.threshold(dst, 0.01 * dst.max(), 255, 0)
dst = np.uint8(dst)

# find centroids
ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv.cornerSubPix(img_gray, np.float32(centroids), (5, 5), (-1, -1), criteria)

# now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]

cv.imwrite('subpixel_messi5.jpg', img)

True